Importing necessary libraraies

In [1]:
import cv2
import os
import imutils 
from imutils import face_utils
import dlib
from scipy.spatial import distance
import vlc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import time
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()  
import warnings
warnings.filterwarnings("ignore")

Importing alert sounds and assigning the thershold values at which the alert sounds will play

In [2]:
alerty=vlc.MediaPlayer('/home/subhranil/Documents/data science/Drowsiness Detection/yawn.mp3')
alerte=vlc.MediaPlayer('/home/subhranil/Documents/data science/Drowsiness Detection/SLEEP.mp3')
alertt=vlc.MediaPlayer('/home/subhranil/Documents/data science/Drowsiness Detection/TAKE REST.mp3')
thresh=0.25
thresh2=0.6
flag=0
flag2=0
flag3=0

PLot the face and derive information

In [3]:
def getFaceDirection(shape, size):
    image_points = np.array([
                                shape[33],    # Nose tip
                                shape[8],     # Chin
                                shape[45],    # Left eye left corner
                                shape[36],    # Right eye right corne
                                shape[54],    # Left Mouth corner
                                shape[48]     # Right mouth corner
                            ], dtype="double")
    
    # 3D model points.
    model_points = np.array([
                                (0.0, 0.0, 0.0),             # Nose tip
                                (0.0, -330.0, -65.0),        # Chin
                                (-225.0, 170.0, -135.0),     # Left eye left corner
                                (225.0, 170.0, -135.0),      # Right eye right corne
                                (-150.0, -150.0, -125.0),    # Left Mouth corner
                                (150.0, -150.0, -125.0)      # Right mouth corner
                            
                            ])
    
    # Camera internals
    
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                            [[focal_length, 0, center[0]],
                            [0, focal_length, center[1]],
                            [0, 0, 1]], dtype = "double"
                            )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
    return(translation_vector[1][0])

Calculating the eye aspect ratio

In [9]:
def eye_aspect_ratio(eye) :
        A=distance.euclidean(eye[1],eye[5])
        B=distance.euclidean(eye[2],eye[4])
        C=distance.euclidean(eye[0],eye[3])
        ear = (A+B)/(2.0*C)
        return ear

Calculating the mouth aspect ratio

In [5]:
def mouth_aspect_ratio(mouth) :
    Am=distance.euclidean(mouth[2],mouth[10])
    Bm=distance.euclidean(mouth[4],mouth[8])
    Cm=distance.euclidean(mouth[0],mouth[6])
    mar = (Am+Bm)/(2.0*Cm)
    return mar

Plotting landmarks on eyes and mouth

In [6]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS['right_eye']
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["mouth"]

Predict the outcome from the model named shape_predictor_68_landmarks and importing it

In [7]:
detect=dlib.get_frontal_face_detector()
predict=dlib.shape_predictor("/home/subhranil/Documents/data science/Drowsiness Detection/shape_predictor_68_face_landmarks.dat")

Detecting yawning, drowsiness and playing alert sound when the aspect ratio vales goes above or below the threshold values

In [8]:
frame_check=20
frame_check2=10
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    size = frame.shape
    gray =cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    subjects = detect(gray,0)
    for subject in subjects :
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)
        #shape2=predict2(gray, subject)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        lips=shape[mStart:mEnd]
        
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        mEAR = mouth_aspect_ratio(lips)
        ear = (leftEAR + rightEAR)/2.0
        
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        mouthHull = cv2.convexHull(lips)
        
        cv2.drawContours(frame, [leftEyeHull],-1, (255,255,255), 1)
        cv2.drawContours(frame, [rightEyeHull],-1, (255,255,255), 1)
        cv2.drawContours(frame, [mouthHull],-1, (255,255,255), 1)
        if mEAR>thresh2 :
            flag2+=1
            print("YAWN",flag2)
            cv2.putText(frame, "YAWNING!!!", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
            cv2.putText(frame, "YAWNING!!!", (10,325),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            alerty.play()

        if ear<thresh :
            flag+=1
            print(flag)
            if flag>=frame_check:
                cv2.putText(frame, "DROWSINESS DETECTED!!!", (150, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
                cv2.putText(frame, "FEELING SLEEPY!!!", (150,325),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                alertt.play()
        elif(ear>thresh):
            flag=0
            flag2=0
        if(ear>thresh and mEAR<thresh2):
            alertt.stop()
            alerty.stop()
          
        #head position  
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
         
        if(getFaceDirection(shape, size)<0):
            cv2.putText(frame, "SIT STRAIGHT!!!", (10, 225),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
        file=open("ear1.csv","a")
        earw=str(ear)
        file.write(earw)
        file.write("\n")
        file=open("mar.csv","a")
        file.write(str(mEAR))
        file.write("\n")

    cv2.imshow('Frame',frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@2.503] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


YAWN 1
YAWN 1
1
YAWN 2
YAWN 1
YAWN 1
1
1
YAWN 1
YAWN 1
YAWN 1
YAWN 1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
YAWN 1
YAWN 1
YAWN 1
YAWN 1
YAWN 1
YAWN 1
YAWN 1
YAWN 1
YAWN 1
YAWN 1
1
2
1
2
1
1
2
1


KeyboardInterrupt: 

Plotting graph for Eye Aspect ratio

In [10]:
colname=['EAR','frames']
d=pd.read_csv('ear1.csv',names=colname)
d.to_csv('ear1.csv',index=False)
d['frames']=d.index
d.iplot(kind='scatter',x="frames",y="EAR",title='EYE_ASPECT_RATIO',
       labels={
           'frames':"FRAMES",
           'EAR':"Aspect Ratio"
       },)

Plotting graph for Mouth Aspect ratio

In [11]:
colname2=['MAR','frames']
d2=pd.read_csv('mar.csv',names=colname2)
d2.to_csv('mar.csv',index=False)
d2['frames']=d2.index
d2.iplot(kind='scatter',x="frames",y="MAR",title='MOUTH_ASPECT_RATIO',
       labels={
           'frames':"FRAMES",
           'EAR':"Aspect Ratio"
       },)

Saving the information in external ear1 and mar csv's

In [12]:
d.drop(['EAR','frames'],axis=1,inplace=True)
d.to_csv('ear1.csv',index=False)
d2.drop(['MAR','frames'],axis=1,inplace=True)
d2.to_csv('mar.csv',index=False)